In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
import os
import shutil

# === Base path where all DR folders are ===
base_dir = "/content/drive/MyDrive/EECE 490 Project/RandSet2"

# === New output folder ===
output_dir = os.path.join(base_dir, "All_Images")
os.makedirs(output_dir, exist_ok=True)

# === Folder names to merge ===
dr_folders = [
    "No DR signs",
    "Mild (or early) NPDR",
    "Moderate NPDR",
    "Severe NPDR",
    "Very Severe NPDR",
    "PDR",
    "Advanced PDR"
]

# === Copy all images without renaming ===
for folder in dr_folders:
    folder_path = os.path.join(base_dir, folder)
    if not os.path.isdir(folder_path):
        print(f"⚠️ Folder not found: {folder_path}")
        continue

    for fname in os.listdir(folder_path):
        if fname.lower().endswith((".jpg", ".jpeg", ".png")):
            src = os.path.join(folder_path, fname)
            dst = os.path.join(output_dir, fname)  # ⚠️ no renaming here!
            shutil.copy2(src, dst)

print(f"\n✅ Done! All images copied without renaming to: {output_dir}")


In [ ]:
import os
import shutil

# === Base paths ===
base_dir = "/content/drive/MyDrive/EECE 490 Project/RandSet2"
input_dir = os.path.join(base_dir, "All_Images")
output_dir = os.path.join(base_dir, "All_Images_JPEG")

# === Create output directory ===
os.makedirs(output_dir, exist_ok=True)

# === Copy & Rename ===
count = 0
for fname in os.listdir(input_dir):
    if fname.lower().endswith((".jpg", ".jpeg")):
        name_wo_ext = os.path.splitext(fname)[0]
        src = os.path.join(input_dir, fname)
        dst = os.path.join(output_dir, name_wo_ext + ".jpeg")
        shutil.copy2(src, dst)
        count += 1

print(f"✅ Copied and renamed {count} image(s) to .jpeg in: {output_dir}")


In [ ]:
import os
import pandas as pd

# === Base path ===
base_dir = "/content/drive/MyDrive/EECE 490 Project/RandSet2"
output_excel = os.path.join(base_dir, "DR_Graded_Metadata.xlsx")

# === DR class mapping ===
dr_mapping = {
    "No DR signs": 0,
    "Mild (or early) NPDR": 1,
    "Moderate NPDR": 2,
    "Severe NPDR": 3,
    "Very Severe NPDR": 4,
    "PDR": 5,
    "Advanced PDR": 6
}

# === Output data list ===
data = []

# === Loop through folders and collect filenames + labels ===
for folder_name, label in dr_mapping.items():
    folder_path = os.path.join(base_dir, folder_name)
    if not os.path.isdir(folder_path):
        print(f"⚠️ Folder not found: {folder_path}")
        continue

    for fname in os.listdir(folder_path):
        if fname.lower().endswith((".jpg", ".jpeg")):
            data.append({
                "Filename": fname,
                "Format": "jpeg",
                "Status": label
            })

# === Save metadata to Excel ===
df = pd.DataFrame(data)
df.to_excel(output_excel, index=False)

print(f"✅ Excel metadata saved to: {output_excel}")


In [ ]:
import os
import pandas as pd

# === Path to original Excel file ===
input_excel = "/content/drive/MyDrive/EECE 490 Project/RandSet2/Manual classification of fundus images with diabetic retinopathy conditions.xlsx"  # Replace with your actual filename
output_excel = "/content/drive/MyDrive/EECE 490 Project/RandSet2/DR_Graded_Metadata.xlsx"

# === DR stage to numeric mapping ===
dr_mapping = {
    "No DR signs": 0,
    "Mild (or early) NPDR": 1,
    "Moderate NPDR": 2,
    "Severe NPDR": 3,
    "Very Severe NPDR": 4,
    "PDR": 5,
    "Advanced PDR": 6
}

# === Load original metadata ===
df = pd.read_excel(input_excel)

# === Map stage names to numeric values ===
df["Status"] = df["Status"].map(dr_mapping)  # Assuming column name is 'Stage' or similar

# === Create a new 'Filename' column with '.jpeg' extension ===
df["Image"] = df["Image"].apply(lambda x: str(x).split('.')[0] + ".jpeg")  # Assuming image column is named 'Image'

# === Save to NEW Excel file ===
df.to_excel(output_excel, index=False)

print(f"✅ New metadata Excel saved to: {output_excel}")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# === Load the Excel file ===
excel_path = "/content/drive/MyDrive/EECE 490 Project/RandSet2/DR_Graded_Metadata.xlsx"
df = pd.read_excel(excel_path)

# === Count distribution ===
grade_counts = df["Status"].value_counts().sort_index()

# === Print results ===
print("📊 DR Grade Distribution:")
for grade, count in grade_counts.items():
    print(f"Grade {grade}: {count} images")


In [ ]:
# === Imports ===
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
import shutil

# === Black Border Remover ===
def remove_black_border(img, threshold=5):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return img
    x, y, w, h = cv2.boundingRect(max(contours, key=cv2.contourArea))
    return img[y:y+h, x:x+w]

# === Auto Flip if Left Eye ===
def auto_flip_if_left_eye(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    h, w = gray.shape
    left = np.mean(gray[:, :w//2])
    right = np.mean(gray[:, w//2:])
    if right > left:
        return cv2.flip(img, 1)  # horizontal flip
    return img

# === Full Preprocessing Pipeline (Resizing moved to end) ===
def preprocess_image_full(img, target_size=(512, 512), blur_thresh=20.0, brightness_thresh=25):
    try:
        if img is None or len(img.shape) != 3 or img.shape[2] != 3:
            return None

        # Blur and brightness checks
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if cv2.Laplacian(gray, cv2.CV_64F).var() < blur_thresh:
            return None
        if np.mean(gray) < brightness_thresh:
            return None

        # Remove black borders
        img = remove_black_border(img)

        # Auto flip if left eye
        img = auto_flip_if_left_eye(img)

        # CLAHE enhancement
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        l = clahe.apply(l)
        img = cv2.cvtColor(cv2.merge((l, a, b)), cv2.COLOR_LAB2BGR)

        # Glare detection
        glare_mask = cv2.inRange(img, (240, 240, 240), (255, 255, 255))
        contours, _ = cv2.findContours(glare_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if any(cv2.contourArea(c) > 500 for c in contours):
            return None

        # Text/annotation filter
        mask_text = cv2.inRange(img, (220, 220, 220), (255, 255, 255))
        if cv2.countNonZero(mask_text) > 3000:
            return None

        # Dark region filter
        if np.mean(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) > 10) < 0.2:
            return None

        # ✅ Resize (critical fix)
        img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)

        # ✅ Validate shape before return
        if img.shape != (512, 512, 3):
            raise ValueError(f"Resized image has invalid shape: {img.shape}")

        # Normalize to [-1, 1]
        return img.astype(np.float32) / 127.5 - 1.0

    except Exception as e:
        print(f"⚠️ Exception during processing: {e}")
        return None

# === Set Input/Output Folders ===
input_folder = "/content/drive/MyDrive/EECE 490 Project/RandSet2/All_Images_JPEG"
output_folder = "/content/drive/MyDrive/EECE 490 Project/RandSet2/All_Images_Cleaned"
os.makedirs(output_folder, exist_ok=True)

# === Run Preprocessing ===
skipped = []
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpeg', '.jpg', '.png'))]
sample_images = []

for fname in tqdm(image_files):
    in_path = os.path.join(input_folder, fname)
    out_path = os.path.join(output_folder, fname)

    img = cv2.imread(in_path)
    if img is None:
        skipped.append(fname)
        continue

    processed = preprocess_image_full(img)
    if processed is None:
        skipped.append(fname)
        continue
    if processed.shape != (512, 512, 3):
        print(f"⚠️ Unexpected size in {fname}: {processed.shape}")
        skipped.append(fname)
        continue

    out_img = ((processed + 1.0) * 127.5).astype(np.uint8)
    cv2.imwrite(out_path, out_img)

    if len(sample_images) < 10 and random.random() < 0.1:
        sample_images.append(cv2.cvtColor(out_img, cv2.COLOR_BGR2RGB))

# === Show Random Previews ===
if sample_images:
    plt.figure(figsize=(20, 5))
    for i, img in enumerate(sample_images):
        plt.subplot(1, len(sample_images), i + 1)
        plt.imshow(img)
        plt.axis("off")
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No sample images to display.")

print(f"\n✅ Done! Preprocessed images saved to: {output_folder} | Skipped {len(skipped)} bad images.")


In [ ]:
import os
import cv2

# === Folder containing images ===
folder_path = "/content/drive/MyDrive/EECE 490 Project/RandSet2/All_Images_Cleaned"

# === Loop through all files and check dimensions ===
for filename in os.listdir(folder_path):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)

        if img is None:
            print(f"❌ Could not read {filename}")
            continue

        h, w = img.shape[:2]
        if h != 512 or w != 512:
            print(f"⚠️ {filename} has size {w}x{h}")


In [ ]:
cleaned_images = [f for f in os.listdir(output_folder) if f.lower().endswith(('.jpeg', '.jpg', '.png'))]
print(f"🧼 Number of cleaned images: {len(cleaned_images)}")


In [ ]:
import os
import pandas as pd

# === Load the combined Excel file ===
excel_path = "/content/drive/MyDrive/EECE 490 Project/RandSet2/DR_Graded_Metadata.xlsx"
df = pd.read_excel(excel_path)

# === Define where your JPEG images are stored ===
output_folder = "/content/drive/MyDrive/EECE 490 Project/RandSet2/All_Images_Cleaned"
kept_images = set(os.listdir(output_folder))

# === Filter rows based on available images ===
df_filtered = df[df['Image'].isin(kept_images)]

# === Count and display label distribution ===
label_counts = df_filtered['Status'].value_counts().sort_index()
count_0 = label_counts.get(0, 0)
count_1 = label_counts.get(1, 0)
count_2 = label_counts.get(2, 0)
count_3 = label_counts.get(3, 0)
count_4 = label_counts.get(4, 0)
count_5 = label_counts.get(5, 0)
count_6 = label_counts.get(6, 0)

print(f"✅ Remaining samples:\n  No_DR (0): {count_0}\n  DR (1): {count_1}\n  DR (2): {count_2}\n  DR (3): {count_3}\n  DR (4): {count_4}\n  DR (5): {count_5}\n  DR (6): {count_6}")

# === Save the filtered dataframe to a new Excel file ===
output_excel_path = "/content/drive/MyDrive/EECE 490 Project/RandSet2/DR_Graded_Cleaned.xlsx"
df_filtered.to_excel(output_excel_path, index=False)

print(f"\n📁 Filtered Excel file saved to: {output_excel_path}")


In [ ]:
import os
import pandas as pd
import shutil

# === Paths ===
base_dir = "/content/drive/MyDrive/EECE 490 Project/RandSet2"
input_excel = os.path.join(base_dir, "DR_Graded_Cleaned.xlsx")
input_images_folder = os.path.join(base_dir, "All_Images_Cleaned")
output_excel = os.path.join(base_dir, "DR_Unbalanced_Split.xlsx")
output_split_folder = os.path.join(base_dir, "Unbalanced_Split")
no_dr_folder = os.path.join(output_split_folder, "NoDR")
dr_folder = os.path.join(output_split_folder, "DR")

# === Step 1: Load Excel and add binary_level column ===
df = pd.read_excel(input_excel)
df["binary_level"] = df["Status"].apply(lambda x: 0 if x == 0 else 1)

# === Step 2: Save new Excel file ===
df.to_excel(output_excel, index=False)
print(f"✅ New Excel file saved to: {output_excel}")

# === Step 3: Create output folders ===
os.makedirs(no_dr_folder, exist_ok=True)
os.makedirs(dr_folder, exist_ok=True)

# === Step 4: Copy images based on binary_level ===
for _, row in df.iterrows():
    fname = row["Image"]
    label = row["binary_level"]

    src_path = os.path.join(input_images_folder, fname)
    dst_path = os.path.join(no_dr_folder if label == 0 else dr_folder, fname)

    if os.path.exists(src_path):
        shutil.copy(src_path, dst_path)

print(f"✅ Images split into {no_dr_folder} and {dr_folder} based on binary labels.")


In [ ]:
import os
import pandas as pd
import shutil

# === Paths ===
filtered_excel_path = "/content/drive/MyDrive/EECE 490 Project/RandSet2/DR_Graded_Cleaned.xlsx"
output_folder = "/content/drive/MyDrive/EECE 490 Project/RandSet2/All_Images_Cleaned"
balanced_excel_path = "/content/drive/MyDrive/EECE 490 Project/RandSet2/DR_Graded_Balanced.xlsx"
balanced_images_folder = "/content/drive/MyDrive/EECE 490 Project/RandSet2/All_Images_Balanced"
os.makedirs(balanced_images_folder, exist_ok=True)

# === Load filtered Excel ===
df_filtered = pd.read_excel(filtered_excel_path)

# === Add binary label column ===
df_filtered['binary_level'] = df_filtered['Status'].apply(lambda x: 0 if x == 0 else 1)

# === Balance the dataset ===
min_count = df_filtered['binary_level'].value_counts().min()
df_balanced = pd.concat([
    df_filtered[df_filtered['binary_level'] == 0].sample(min_count, random_state=42),
    df_filtered[df_filtered['binary_level'] == 1].sample(min_count, random_state=42)
]).reset_index(drop=True)

# === Save balanced Excel ===
df_balanced.to_excel(balanced_excel_path, index=False)
print(f"✅ Balanced Excel file saved to: {balanced_excel_path}")

# === Copy images to Balanced_Images folder ===
for fname in df_balanced['Image']:
    src = os.path.join(output_folder, fname)
    dst = os.path.join(balanced_images_folder, fname)
    if os.path.exists(src):
        shutil.copy(src, dst)

print(f"✅ {len(df_balanced)} images copied to: {balanced_images_folder}")


In [ ]:
# === Count binary label distribution in balanced dataset ===
binary_counts = df_balanced['binary_level'].value_counts().sort_index()
count_0 = binary_counts.get(0, 0)
count_1 = binary_counts.get(1, 0)

print(f"📊 Class Distribution in Balanced excel:\n  No_DR (0): {count_0}\n  DR (1): {count_1}")


In [ ]:
import os
import pandas as pd
import shutil

# === Paths ===
balanced_excel_path = "/content/drive/MyDrive/EECE 490 Project/RandSet2/DR_Graded_Balanced.xlsx"
balanced_images_folder = "/content/drive/MyDrive/EECE 490 Project/RandSet2/All_Images_Balanced"
split_folder = "/content/drive/MyDrive/EECE 490 Project/RandSet2/All_Split_Balanced"
dr_folder = os.path.join(split_folder, "DR")
no_dr_folder = os.path.join(split_folder, "NoDR")

# === Create target folders ===
os.makedirs(dr_folder, exist_ok=True)
os.makedirs(no_dr_folder, exist_ok=True)

# === Load balanced Excel ===
df_balanced = pd.read_excel(balanced_excel_path)

# === Copy images to corresponding folders ===
for _, row in df_balanced.iterrows():
    fname = row['Image']
    label = row['binary_level']

    src = os.path.join(balanced_images_folder, fname)
    dst = os.path.join(dr_folder if label == 1 else no_dr_folder, fname)

    if os.path.exists(src):
        shutil.copy(src, dst)

print("✅ Images split into DR and NoDR folders successfully!")
